In [2]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head, process_data
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [3]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [4]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER, FWHM, R, S
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER	FWHM	R	S
1	600	405	45	M5	23.0	8.0	15.9
2	600	405	53	P3	25.0	19.7	3.8
3	650	452	32	K4	12.5	11.9	17.8
4	650	452	40	K5	15.0	18.0	8.8
5	750	528	24	L3	7.0	13.8	78.5
6	750	526	30	Q4	9.5	22.0	30.7
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [5]:
con.commit()

In [6]:
# 
columns = 'MASTER,  AVG(DIP), AVG(INP), AVG(INP/IBGPX), AVG(CON), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	 AVG(DI	AVG(INP	AVG(INP	AVG(CON	AVG(INP	AVG(IBG
L3	740.00	18.53	54.94	3.31	1.15	0.3505
Q4	727.88	25.47	72.46	5.13	1.79	0.3870
K5	658.00	14.79	48.99	5.44	1.64	0.3078
K4	647.00	6.18	24.67	3.32	0.9056	0.2691
P3	615.00	5.25	20.50	7.27	1.81	0.2621
M5	606.00	2.54	11.68	6.60	1.42	0.2196


In [11]:
# 
columns = 'NP, FILE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'Q4' AND DATE='20-04-20' AND WAVELENGTH = 732 AND EXP_TYPE = '-'
ORDER BY NP DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

NP	FILE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP
80	raw_02_	24.29	9.28	11.33	225.36	25.61	19.26	0.2904	-
60	raw_25_	37.79	12.75	5.39	481.71	27.55	20.75	0.2537	-
40	raw_10_	33.70	12.84	5.93	432.52	33.27	27.78	0.2710	-
---
3


In [26]:
# 
columns = 'MASTER, NP, WAVELENGTH, FOLDER, FILE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'P3'
ORDER BY CON DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

MASTER	NP	WAVELEN	FOLDER	FILE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP
P3	80	615	20_02_2	raw_08_	4.83	5.63	9.08	27.17	4.61	2.02	0.2231	-
P3	60	612	20_02_2	raw_16_	4.96	5.39	8.28	26.76	4.66	2.03	0.2225	-
P3	80	612	20_02_2	raw_29_	5.11	6.36	8.16	32.51	4.91	2.18	0.2032	sh
P3	80	614	20_02_2	raw_09_	5.23	5.52	8.03	28.83	4.36	2.37	0.2244	-
P3	80	612	20_02_2	raw_10_	5.06	5.98	7.86	30.26	4.53	2.27	0.1912	-
P3	80	622	20_02_2	raw_06_	5.55	5.11	7.80	28.36	5.34	2.83	0.2645	-
P3	80	612	20_02_2	raw_32_	5.11	6.11	7.43	31.24	4.54	2.02	0.2021	strep
P3	80	628	20_02_2	raw_03_	5.86	5.48	6.86	32.14	6.73	4.08	0.3297	-
P3	80	618	20_02_2	raw_07_	6.58	6.58	6.82	43.26	5.85	1.92	0.2803	-
P3	80	610	20_02_2	raw_11_	6.25	6.53	6.63	40.83	5.63	2.33	0.2363	-
P3	80	624	20_02_2	raw_05_	5.50	5.97	6.52	32.84	5.26	2.83	0.2794	-
P3	80	602	20_02_2	raw_14_	4.93	5.89	6.51	28.99	6.21	2.97	0.3352	-
P3	80	626	20_02_2	raw_04_	5.66	4.57	6.48	25.87	5.68	3.98	0.4105	-
P3	80	606	20_02_2	raw_13_	5.33	6.04	6.32	32.18	5.42	2.60	0.2714	-
P3	

In [12]:
folder = '20_04_20_Q4/'
file = 'raw_17_1'

columns = 'COUNT(MEAS.ID), MEAS.ID'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID 
WHERE FOLDER = '{}' AND FILE = '{}'
;
   """.format(columns, folder, file)
                  )

In [26]:
[d for d in data][0]

(3, 79)

In [20]:
b

In [28]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS, AR_MIN, AR_COND, exp_type'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=732
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS	AR_MIN	AR_COND	exp_type
80	732	60.0	0.001	[2]	10	2	-
60	732	59.0	0.001	[2]	10	2	-
60	732	46.0	0.001	[2]	10	2	sh
40	732	12.0	0.001	[1.1]	2	2	-
80	732	100.0	0.001	[2]	2	2	-
60	732	20.0	0.001	[2]	2	2	-
80	732	40.0	0.001	[2]	2	2	strep
147	732	8.0	0.001	[2]	2	2	ps
40	732	9.0	0.001	[2]	6	2	-
80	732	31.0	0.001	[2]	6	2	sh


In [8]:
con.execute("""
UPDATE 'measurements' SET NOTES = 'weak' WHERE COUNT = 165
""")

In [215]:
meas.commit()

In [51]:
con.close()